In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.all import (
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    LinearSystem
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState

meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [80]:
def UprightState():
    state = AcrobotState()
    state.set_theta1(np.pi)
    state.set_theta2(0.0)
    state.set_theta1dot(0.0)
    state.set_theta2dot(0.0)
    return state

The singular values of the controllability matrix are: [5.11718692e+02 1.23084288e+02 8.06819874e-02 7.15326677e-02]


In [64]:
from control.AcrobotControllers import AcrobotiLQRController, AcrobotLQRController
from control.UtilLeafSystems import NonConstantVectorSource

builder = DiagramBuilder()
acrobot = builder.AddSystem(AcrobotPlant())
# saturation = builder.AddSystem(
#     Saturation(min_value=[-10], max_value=[10])
#     )
# builder.Connect(
#     saturation.get_output_port(0), acrobot.get_input_port(0)
#     )
wrapangles = WrapToSystem(4)
wrapangles.set_interval(0, 0, 2.0 * np.pi)
wrapangles.set_interval(1, -np.pi, np.pi)
wrapto = builder.AddSystem(wrapangles)
builder.Connect(acrobot.get_output_port(0), wrapto.get_input_port(0))


goal_state_source = builder.AddSystem(NonConstantVectorSource(4))

Q = np.diag(np.concatenate(([0.0] * 2, [0.01] * 2)))
Qf = np.diag(np.concatenate(([100.0] * 2, [100.0] * 2)))
R = np.diag([0.01])
dt = 0.05
N = 40
dt = 0.05
iLQRParams = {
    "acrobot" : acrobot,
    "Q" : Q,
    "R" : R,
    "Qf" : Qf,
    "N" : N,
    "dt" : dt,
    "max_iter" : 50,
    "max_linesearch_iters": 20,
    "mode": "discrete"
}

controller = builder.AddSystem(AcrobotiLQRController(**iLQRParams))
# controller = builder.AddSystem(AcrobotLQRController(**iLQRParams))

builder.Connect(
    wrapto.get_output_port(0), controller.get_input_port(0)
    )
builder.Connect(
    controller.get_output_port(0), acrobot.get_input_port(0)
    )
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
    )
 # Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()


def simulate(diagram):
    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    # Simulate
    simulator.set_target_realtime_rate(1.0)
    duration = 10.0
    ref_state = np.array([np.pi, 0., 0., 0.])
    goal_state_source.SetState(ref_state)
    for i in range(5):
        context.SetTime(0.0)
        context.SetContinuousState(
            ref_state
            + 0.5
            * np.random.randn(
                4,
            )
        )
        # context.SetContinuousState(np.array([np.pi/3, , 0.0, 0.0]))
        simulator.Initialize()
        simulator.AdvanceTo(duration)

simulate(diagram)

Init utraj


/usr/lib/python3/dist-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/home/malkstik/mnt/control/models/acrobot.py:116: RuntimeWarning: overflow encountered in matmul
  state_cost = 0.5*(xerr).T @ self.Q @ (xerr)
/home/malkstik/mnt/control/models/acrobot.py:117: RuntimeWarning: overflow encountered in matmul
  action_cost = 0.5*(uerr).T @ self.R @ (uerr)


LinAlgError: Array must not contain infs or NaNs

In [71]:
from IPython.display import SVG, display, clear_output, Markdown
from control.models.acrobot import Acrobot2DModel
from control.iLQRController import iLQR

Q = np.diag(np.concatenate(([1.0] * 2, [0.1] * 2)))
Qf = np.diag(np.concatenate(([100.0] * 2, [100.0] * 2)))
R = np.diag([0.01])
dt = 0.05
N = 40

iLQRParams = {
    "Q" : Q,
    "R" : R,
    "Qf" : Qf,
    "N" : N,
    "dt" : dt,
    "max_iter" : 20,
    "regu_init" : 1000.0,
    "min_regu": 0.001,
    "max_regu": 10000,
    "max_linesearch_iters": 10,
    "integration": "euler",
}
plant = Acrobot2DModel(**iLQRParams)
controller = iLQR(plant, **iLQRParams)
goal_state  = np.array([np.pi, 0., 0., 0.])
# goal_state  = np.array([0., 0., 0., 0.])
current_state = np.array([0., 0., 0., 0.])

nx, nu = plant.get_dims()
uref = np.zeros(nu, dtype = np.float64)
xtraj = np.zeros((N, nx))
utraj = np.zeros((N-1, nu))
controller.model.set_references(goal_state, uref)

# print(controller.model.discrete_dynamics(goal_state, 0))
# print(controller.model.xdot_fun(goal_state, 0))
state_error = 0
time = 0
while np.linalg.norm(state_error) > 1e-4 or time == 0:
    if not utraj.any():
        utraj[:] = np.random.randn(N-1, nu)*1.0
    else:
        # Use previous utraj as initial guess with shifting 
        # forward a single timestep, 
        utraj[:-1, :] = utraj[1:,:] #+  np.random.randn(N-2, nu)

    xtraj, utraj = controller.run_ilqr(current_state, utraj)
    current_state = xtraj[1, :]
    u = utraj[0, :]
    current_state = np.squeeze(current_state)

    state_error = current_state - goal_state
    state_error[:2] /= np.pi
    terminal_state_error = xtraj[-1, :] - goal_state
    terminal_state_error[:2] /= np.pi
    # print(xtraj)
    clear_output(wait=True)
    display(Markdown(f"**t = {time:4.4f}s:**\n```\nError: {state_error}\n```\n**Control:**\n```\n{u}\n```\n**Terminal State Err:**\n```\n{terminal_state_error}"))
    time += dt

**t = 5.9500s:**
```
Error: [-4.37274180e-06 -1.42099503e-05  3.27897707e-05  8.91836754e-05]
```
**Control:**
```
[0.00018036]
```
**Terminal State Err:**
```
[-2.75049665e-07 -1.46752638e-07 -2.70006483e-07 -1.33102914e-07]